# Find and update all add'l USCDI elements in StructureDefinitions

prerequisite - need a completed local build

- for hard coded path search all SD for all element with Add'l USCDI tag
- associated url with title
- update csv in IG data file
  - need manually add USCDI mapping so
- use liquid template to geenerate table in IG

### Imports and Constants

In [1]:
from pathlib import Path
from json import loads, dumps
from yaml import load, SafeLoader
import pandas as pd

ig_path = '/Users/ehaas/Documents/FHIR/US-Core/'
ig_base_url = 'http://hl7.org/fhir/us/core/'  # US Core don't forget the trailing /
fhir_base = 'http://hl7.org/fhir/' # canonical url for FHIR don't forget the trailing /
addl_csv = 'input/data/additional-uscdi-requirements.csv'
out_path = Path(r'/Users/ehaas/Documents/FHIR/test-stuff/input/data')


### get the CSV and create DF

In [2]:
file_path = Path() / ig_path / addl_csv
df = pd.read_csv(file_path)
df['combo'] = df['Profile'] +'-'+ df['FHIR_Element']
df

,Is_New,Addl_USCDI,Profile,FHIR_Element,combo
0,NaN,Contact Detail,US Core Patient Profile,Patient.telecom,US Core Patient Profile-Patient.telecom
1,NaN,A Communication Language,US Core Patient Profile,Patient.communication,US Core Patient Profile-Patient.communication
2,True,An Interpreter Required Flag,US Core Patient Profile,Patient.extension:interpreterRequired,US Core Patient Profile-Patient.extension:inte...
3,True,An Interpreter Required Flag,US Core Encounter Profile,Encounter.extension:interpreterRequired,US Core Encounter Profile-Encounter.extension:...
4,NaN,A Race,US Core Patient Profile,Patient.extension:race,US Core Patient Profile-Patient.extension:race
5,NaN,An Ethnicity,US Core Patient Profile,Patient.extension:ethnicity,US Core Patient Profile-Patient.extension:ethn...
6,NaN,A Tribal Affiliation,US Core Patient Profile,Patient.extension:tribalAffiliation,US Core Patient Profile-Patient.extension:trib...
7,NaN,A Sex,US Core Patient Profile,Patient.extension:sex,US Core Patient Profile-Patient.extension:sex
8,True,Sex Parameter For Clinical Use,US Core Patient Profile,Patient.extension:sexParameterForClinicalUse,US Core Patient Profile-Patient.extension:sexP...
9,NaN,Gender Identity,US Core Patient Profile,Patient.extension:genderIdentity,US Core Patient Profile-Patient.extension:gend...


### Function to to find the elements

fetch csv 

- aglorithm:
  
  - cycles through all SD's
  - cycle through all elements
  - if extension =  `http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement` then save in list

In [3]:
my_path = Path() / ig_path / 'input' / 'resources-yaml'
for i in sorted(my_path.glob('Struct*.yml')):
  # print(i)
  obj = load(i.read_text(), Loader=SafeLoader) #dict
  # print(obj['title'])
  # print(f"US Core Profile = {obj['title']}")
  if  obj['type'] == 'Extension':
      continue
  # print(f"US Core Profile = {obj['title']}")
  for e in obj['differential']['element']:
      
    try:
      if e["extension"] and 'http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement' in [x['url'] for x in e["extension"]]:
          pass            
    except KeyError as e:
        # print(f"======== no {e} in element !========")
        pass
    else:
        if f"{obj['title']}-{e['id']}" not in df.combo.values:
          print(obj['title'], e['id'])
          new_row = [True,None,obj['title'],e['id'], f"{obj['title']}-{e['id']}"]
          df.loc[len(df)] = new_row


In [4]:
df

,Is_New,Addl_USCDI,Profile,FHIR_Element,combo
0,NaN,Contact Detail,US Core Patient Profile,Patient.telecom,US Core Patient Profile-Patient.telecom
1,NaN,A Communication Language,US Core Patient Profile,Patient.communication,US Core Patient Profile-Patient.communication
2,True,An Interpreter Required Flag,US Core Patient Profile,Patient.extension:interpreterRequired,US Core Patient Profile-Patient.extension:inte...
3,True,An Interpreter Required Flag,US Core Encounter Profile,Encounter.extension:interpreterRequired,US Core Encounter Profile-Encounter.extension:...
4,NaN,A Race,US Core Patient Profile,Patient.extension:race,US Core Patient Profile-Patient.extension:race
5,NaN,An Ethnicity,US Core Patient Profile,Patient.extension:ethnicity,US Core Patient Profile-Patient.extension:ethn...
6,NaN,A Tribal Affiliation,US Core Patient Profile,Patient.extension:tribalAffiliation,US Core Patient Profile-Patient.extension:trib...
7,NaN,A Sex,US Core Patient Profile,Patient.extension:sex,US Core Patient Profile-Patient.extension:sex
8,True,Sex Parameter For Clinical Use,US Core Patient Profile,Patient.extension:sexParameterForClinicalUse,US Core Patient Profile-Patient.extension:sexP...
9,NaN,Gender Identity,US Core Patient Profile,Patient.extension:genderIdentity,US Core Patient Profile-Patient.extension:gend...


### save updated CSV

In [5]:
df.to_csv(file_path, index=False)